In [1]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/tuwen/8'
hour = 2020050822
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df['mark'] = 'tuwen'
# info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# info_path = f'{info_dir}/{hour}/valid.csv'
# df2 = pd.read_csv(info_path)
# df2['dur'] = df.duration
# df2['mark'] = 'video'
# df = pd.concat([df, df2])
df = df[df.product_data=='sgsapp']
df.head()

/search/odin/publicData/CloudS/libowei/rank_online/infos/tuwen/8/2020050822/valid.csv


,doc_topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,id,impression_time,...,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur,mark
0,0,4,1588764222,2,7751987992946443570,3028,20506l0l42osgG,0,721a866655031185162\t20506l0l42osgG,1588945094,...,0,1.0,0,sgsapp,0.023042,0.088624,0.412387,2020050822,0,tuwen
1,0,16,1588867796,1,-1694197970914465223,3039,open_doc_prod28265848,64,35c3864826043423386\topen_doc_prod28265848,1588944890,...,1,1.0,0,sgsapp,0.127533,0.633218,0.123271,2020050822,64,tuwen
2,0,16,1588640321,7,-4696901797553807890,3116,open_doc_prod28047474,0,3f65860469041174513\topen_doc_prod28047474,1588948145,...,0,1.0,0,sgsapp,0.054822,0.234560,0.233364,2020050822,0,tuwen
3,0,0,1588314032,6,-7499005563532932609,3019,20501f0l3qu8D1,0,c85e1588292268995|8a4770\t20501f0l3qu8D1,1588944951,...,0,1.0,0,sgsapp,0.033811,0.160258,0.237362,2020050822,0,tuwen
4,0,2,1588922950,1,5004843507350979974,3116,20508f0T39QH90,0,b89e864395038608971\t20508f0T39QH90,1588946523,...,0,1.0,0,sgsapp,0.013509,0.085229,0.206792,2020050822,0,tuwen


In [4]:
df['click'] = (df.dur != 0).astype(int)

In [5]:
# kv = KV()
# titles = kv.get_titles(list(set(df.docid.values)))
# df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

In [6]:
pd.set_option('display.max_rows', 1000)
pd.set_option("display.precision", 8)

In [7]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [8]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [9]:
mean_duration = df[df.dur > 0].dur.mean()

In [10]:
mean_duration

93.39725693648246

In [11]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [12]:
import math
df.pred = df.pred.apply(lambda x: math.sqrt(x))

In [13]:
cold_mids = set(df[df.cold==True].mid.values)

In [14]:
len(set(df.mid.values))

111499

In [15]:
len(cold_mids)

5800

In [16]:
# def debug(mid=None, cold=False):
#   if mid is None:
#     if not cold:
#       mid = df.mid.sample(1).values[0]
#     else:
#       mid = df[df.mid.isin(cold_mids)].mid.sample(1).values[0]
#   print('mid:', mid)
#   cols = ['position', 'title', 'mark', 'dur', 'ori_lr_score', 'pred', 'pred_click', 'pred_dur', 'docid']
# #   if 'hour' in df.columns:
# #     cols += ['hour']
#   d = df[df.mid==mid]
#   start_time = d.impression_time.min()
#   d['time'] = d.impression_time.apply(lambda x: int((x - start_time) / 60))
# #   cols += ['time']
#   d = d.sort_values(['impression_time', 'position'])[cols].reset_index(drop=True)
# #   return d.sort_values(['ori_lr_score'], ascending=False)[cols].reset_index(drop=True)
#   return d

In [17]:
# debug()

In [18]:
len(set(df.impression.values)) 

364616

In [19]:
len(set(df.mid.values))

111499

In [20]:
len(set(df.impression.values))  / len(set(df.mid.values))

3.2701279832106116

In [21]:
len(set(df[df.dur!=0].mid.values))

99042

In [22]:
len(set(df[df.dur!=0].mid.values)) / len(set(df.mid.values))

0.888277024906053

In [23]:
d_tuwen = df[df.mark=='tuwen']

In [24]:
from projects.feed.rank.src.evaluate import evaluate_df as eval
import melt 

In [25]:
melt.init_flags()

In [31]:
df.ol_pred =  df.ol_pred_click ** 1.5 * df.ol_pred_dur ** 0.5

In [32]:
result = eval(df, group_by_impression=True, key='ol_pred', eval_click=False, 
              eval_duration=False, eval_quality=False, eval_cold=False)
result 

{'auc': 0.7547184982326425,
 'mse': 0.0074455728247434265,
 'mae': 0.03856948710116614,
 'time_auc': 0.7379208411442106,
 'mean': 0.0291790570760508,
 'loss/click': 0.6279526451823096,
 'loss/dur': array(0.10779821, dtype=float32),
 'stats/pos_ratio': 0.16470850074916119,
 'vtime_auc': nan,
 'finish_auc': 0.785833260897298,
 'rcr_auc': 0.73556249286443,
 'ptime_auc': 0.5270494734037122,
 'stats/num_instances': 2197124,
 'stats/click_ratio': 0.16470850074916119,
 'stats/time_per_show': 0.24864667019822026,
 'stats/time_per_click': 0.025943682482356237,
 'stats/time_per_user': 0.02482401195900466,
 'stats/vtime_per_show': 0.0,
 'stats/vtime_per_click': 0.0,
 'stats/vtime_per_user': 0.0,
 'click/time_auc': 0.5662657950562391,
 'click/loss/dur': array(0.5486041, dtype=float32),
 'click/mse': 0.03825483490324917,
 'click/rmse': 0.8044115675627794,
 'click/mae': 0.11856340181396735,
 'click/mean': 0.05867625523607259,
 'group/auc': 0.6713282952093808,
 'group/auc2': 0.6829602728614685,
 'gro